# Agenda

1. Pivot tables
2. Stack, unstack, melt
3. Joins
4. `filter` on columns
5. Cleaning data

In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame

In [2]:
df = DataFrame(np.random.randint(0, 1000, [4,5]),
              index=list('abcd'),
              columns=list('vwxyz'))
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466


In [3]:
df.loc['e'] = [1,2,3,4,5]
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466
e,1,2,3,4,5


In [4]:
new_row = {'w':20, 'x':30, 'z':50}
df.loc['f'] = new_row

In [5]:
df

,v,w,x,y,z
a,906.0,704,539,445.0,618
b,692.0,477,543,649.0,594
c,853.0,732,548,193.0,643
d,501.0,406,614,962.0,466
e,1.0,2,3,4.0,5
f,NaN,20,30,NaN,50


In [6]:
df['w'] = [10, 20, 30, 40, 50, 60]
df

,v,w,x,y,z
a,906.0,10,539,445.0,618
b,692.0,20,543,649.0,594
c,853.0,30,548,193.0,643
d,501.0,40,614,962.0,466
e,1.0,50,3,4.0,5
f,NaN,60,30,NaN,50


In [7]:
df['u'] = {'a':100, 'c':300, 'e':500}

In [8]:
df

,v,w,x,y,z,u
a,906.0,10,539,445.0,618,100.0
b,692.0,20,543,649.0,594,NaN
c,853.0,30,548,193.0,643,300.0
d,501.0,40,614,962.0,466,NaN
e,1.0,50,3,4.0,5,500.0
f,NaN,60,30,NaN,50,NaN


# Grouping

If we have a data frame with:
- One categorical column
- One numeric column

We can use `groupby` to calculate an aggregation method once per category, for all numeric rows matching.

In [9]:
df = pd.read_csv('taxi.csv')
df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0    25.570000
1    17.368569
2    18.406306
3    17.994704
4    18.881648
5    17.211269
6    17.401355
Name: total_amount, dtype: float64

In [10]:
# we can do a 2D grouping

df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

# Pivot table

A pivot table is a 2-dimensional grouping!  It's similar to our multi-index, but uses a table

- We need one categorical column -- this will be the index, for the rows
- A second categorical column - this will be for the columns
- A numeric column
- Aggregation method

In [11]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,25.57,16.941386,19.076807,19.002803,20.518657,20.466667,NaN
2,NaN,17.904989,17.855770,17.359076,17.927913,17.192379,17.401355


In [12]:
df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

In [13]:
df.pivot(index='VendorID', columns='passenger_count', values='total_amount')

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='mean')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,25.57,16.941386,19.076807,19.002803,20.518657,20.466667,NaN
2,NaN,17.904989,17.855770,17.359076,17.927913,17.192379,17.401355


In [16]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='max')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,36.39,252.35,102.35,73.55,73.84,27.30,NaN
2,NaN,137.59,138.84,74.46,72.92,102.11,83.12


In [18]:
# what if we want both mean + standard deviation?
df.pivot_table(columns='VendorID', index='passenger_count', values='total_amount',
              aggfunc=['mean', 'std'])

mean                   std           
VendorID                 1          2          1          2
passenger_count                                            
0                25.570000        NaN  15.301791        NaN
1                16.941386  17.904989  15.369459  15.167490
2                19.076807  17.855770  16.049125  15.233530
3                19.002803  17.359076  15.389968  12.540837
4                20.518657  17.927913  15.845568  14.630713
5                20.466667  17.192379   6.751543  14.064202
6                      NaN  17.401355        NaN  13.363827

In [19]:
# what if we want to look at more than one value column?
df.pivot_table(columns='VendorID', index='passenger_count', values=['total_amount', 'trip_distance'],
              aggfunc='mean')

total_amount            trip_distance          
VendorID                   1          2             1         2
passenger_count                                                
0                  25.570000        NaN      4.600000       NaN
1                  16.941386  17.904989      2.956456  3.262967
2                  19.076807  17.855770      3.452027  3.328849
3                  19.002803  17.359076      3.588535  3.187189
4                  20.518657  17.927913      3.952239  3.440522
5                  20.466667  17.192379      4.933333  3.172553
6                        NaN  17.401355           NaN  3.170976

In [23]:
# what if we want to look at more than one category for columns?
df.pivot_table(columns=['VendorID', 'payment_type'], index='passenger_count', values='total_amount',
              aggfunc='mean')

VendorID                 1                                        2  \
payment_type             1          2          3       4          1   
passenger_count                                                       
0                25.570000        NaN        NaN     NaN        NaN   
1                19.673469  12.698253  13.215172  11.675  20.147390   
2                21.338436  16.006464  42.480000     NaN  20.623066   
3                19.225065  18.763418        NaN  20.800  18.548926   
4                24.138387  17.401667        NaN     NaN  21.539123   
5                      NaN  20.466667        NaN     NaN  19.634750   
6                      NaN        NaN        NaN     NaN  18.980288   

VendorID                              
payment_type             2    3    4  
passenger_count                       
0                      NaN  NaN  NaN  
1                14.510453 -5.3 -7.8  
2                13.905152  NaN  NaN  
3                15.586200  NaN  NaN  
4                14.378966  NaN  NaN  
5                13.225076  NaN  NaN  
6                15.361491  NaN  NaN

In [22]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [24]:
# everything can be a list!
df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

mean                                     \
                             total_amount            trip_distance             
VendorID                                1          2             1         2   
passenger_count payment_type                                                   
0               1               25.570000        NaN      4.600000       NaN   
1               1               19.673469  20.147390      3.346904  3.486413   
                2               12.698253  14.510453      2.358766  2.925338   
                3               13.215172  -5.300000      2.000000  0.465000   
                4               11.675000  -7.800000      1.925000  0.890000   
2               1               21.338436  20.623066      3.461350  3.751722   
                2               16.006464  13.905152      3.346388  2.725152   
                3               42.480000        NaN     11.700000       NaN   
3               1               19.225065  18.548926      3.110390  3.278926   
                2               18.763418  15.586200      4.054430  3.050500   
                4               20.800000        NaN      3.600000       NaN   
4               1               24.138387  21.539123      4.348387  4.059298   
                2               17.401667  14.378966      3.611111  2.832414   
5               1                     NaN  19.634750           NaN  3.518750   
                2               20.466667  13.225076      4.933333  2.610203   
6               1                     NaN  18.980288           NaN  3.178606   
                2                     NaN  15.361491           NaN  3.161118   

                                      std                                     
                             total_amount            trip_distance            
VendorID                                1          2             1         2  
passenger_count payment_type                                                  
0               1               15.301791        NaN      4.666905       NaN  
1               1               17.401710  16.330567      4.120243  4.238130  
                2               10.068471  12.397838      3.495789  3.973524  
                3               15.363039   2.828427      3.300000  0.657609  
                4                6.750000        NaN      1.960230       NaN  
2               1               17.341582  16.457943      4.038718  4.584112  
                2               13.427005  12.278545      4.502270  3.465173  
                3               27.061360        NaN      9.134002       NaN  
3               1               14.268286  13.073727      3.406595  3.529512  
                2               16.588856  11.539061      5.085722  3.363113  
                4                     NaN        NaN           NaN       NaN  
4               1               18.045435  17.046583      4.541429  4.831724  
                2               13.143542  10.806051      3.871048  4.002527  
5               1                     NaN  15.734323           NaN  4.261502  
                2                6.751543   9.604873      2.926317  3.393812  
6               1                     NaN  13.856405           NaN  3.499148  
                2                     NaN  12.447223           NaN  4.083165

In [25]:
pt = df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

In [29]:
# passenger_count 4
# payment_type 1

# function mean
# column total_amount
# vendor ID 2

pt.loc[
   (4, 1)   # row selector
,
   # column selector
   ('mean', 'total_amount', 2)
]

21.539122807017545

In [30]:
# I want payment type 1 for all passenger counts

pt.xs(1, level='payment_type')

mean                                             std  \
                total_amount            trip_distance           total_amount   
VendorID                   1          2             1         2            1   
passenger_count                                                                
0                  25.570000        NaN      4.600000       NaN    15.301791   
1                  19.673469  20.147390      3.346904  3.486413    17.401710   
2                  21.338436  20.623066      3.461350  3.751722    17.341582   
3                  19.225065  18.548926      3.110390  3.278926    14.268286   
4                  24.138387  21.539123      4.348387  4.059298    18.045435   
5                        NaN  19.634750           NaN  3.518750          NaN   
6                        NaN  18.980288           NaN  3.178606          NaN   

                                                    
                           trip_distance            
VendorID                 2             1         2  
passenger_count                                     
0                      NaN      4.666905       NaN  
1                16.330567      4.120243  4.238130  
2                16.457943      4.038718  4.584112  
3                13.073727      3.406595  3.529512  
4                17.046583      4.541429  4.831724  
5                15.734323           NaN  4.261502  
6                13.856405           NaN  3.499148

In [31]:
# we can use xs on columns, too!

pt.xs('trip_distance', level=1, axis='columns')

mean                 std          
VendorID                              1         2         1         2
passenger_count payment_type                                         
0               1              4.600000       NaN  4.666905       NaN
1               1              3.346904  3.486413  4.120243  4.238130
                2              2.358766  2.925338  3.495789  3.973524
                3              2.000000  0.465000  3.300000  0.657609
                4              1.925000  0.890000  1.960230       NaN
2               1              3.461350  3.751722  4.038718  4.584112
                2              3.346388  2.725152  4.502270  3.465173
                3             11.700000       NaN  9.134002       NaN
3               1              3.110390  3.278926  3.406595  3.529512
                2              4.054430  3.050500  5.085722  3.363113
                4              3.600000       NaN       NaN       NaN
4               1              4.348387  4.059298  4.541429  4.831724
                2              3.611111  2.832414  3.871048  4.002527
5               1                   NaN  3.518750       NaN  4.261502
                2              4.933333  2.610203  2.926317  3.393812
6               1                   NaN  3.178606       NaN  3.499148
                2                   NaN  3.161118       NaN  4.083165

# Exercise: Olympic pivot tables

1. Load the Olympic data (`olympic_athlete_events.csv`) into a data frame. Keep only the rows for the teams from : Israel, US, UK, France, India, China.
2. Create a pivot table of teams vs. sport, average height, where the year is equal to or after 1980, and the sport has `i` in its name.
3. Show a pivot table of teams vs. sport, min + max weight. 
4. Show a pivot table of teams vs. sport, min and max weight + height.
5. Show a pivot table of years vs. teams, number of competitors.  Retrieve the number for Israel in each year.

In [32]:
pt = df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

In [34]:
df = pd.read_csv('olympic_athlete_events.csv',
                usecols=['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'])
df.columns

Index(['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'], dtype='object')

In [39]:
# keep only rows from 1980-
df = df.loc[df['Year'] >= 1980]

In [43]:
# keep only from countries: Israel, US, UK, France, India, China

df = df.loc[df['Team'].isin(['United States', 'Israel', 'United Kingdom', 'France', 'India', 'China'])]
df.head(10)

,Name,Age,Height,Weight,Team,Year,Sport
0,A Dijiang,24.0,180.0,80.0,China,1992,Basketball
1,A Lamusi,23.0,170.0,60.0,China,2012,Judo
10,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
11,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
12,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
13,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
14,Per Knut Aaland,33.0,188.0,75.0,United States,1994,Cross Country Skiing
15,Per Knut Aaland,33.0,188.0,75.0,United States,1994,Cross Country Skiing
16,Per Knut Aaland,33.0,188.0,75.0,United States,1994,Cross Country Skiing
17,Per Knut Aaland,33.0,188.0,75.0,United States,1994,Cross Country Skiing


In [44]:
df['Team'].value_counts()

Team
United States    8858
France           5550
China            4874
India             759
Israel            474
Name: count, dtype: int64

In [49]:
# keep only sports with "i" in the name

df = df.loc[df['Sport'].str.contains('i')]
df.head()

,Name,Age,Height,Weight,Team,Year,Sport
10,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
11,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
12,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
13,Per Knut Aaland,31.0,188.0,75.0,United States,1992,Cross Country Skiing
14,Per Knut Aaland,33.0,188.0,75.0,United States,1994,Cross Country Skiing


In [50]:
# 2. Create a pivot table of teams vs. sport, average height, where the year is equal to or after 1980, and the sport has `i` in its name.

# index = Sport
# columns = Team
# values = Height
# aggregation method =  mean

df.pivot_table(index='Sport',
              columns='Team',
              values='Height',
              aggfunc='mean')




Team,China,France,India,Israel,United States
Sport,,,,,
Alpine Skiing,171.166667,173.445255,167.600000,181.000000,174.090379
Athletics,174.985646,177.066901,171.186047,177.166667,178.065041
Badminton,176.448980,173.650000,175.000000,171.000000,170.818182
Biathlon,167.177215,173.540650,NaN,NaN,173.824645
Bobsleigh,NaN,182.368421,NaN,NaN,NaN
Boxing,176.500000,175.033333,171.484848,171.000000,175.989583
Canoeing,179.366972,179.234973,NaN,180.600000,178.350318
Cross Country Skiing,169.520408,172.902834,167.666667,NaN,173.645503
Curling,172.411765,177.100000,NaN,NaN,174.510638


In [52]:
# 3. Show a pivot table of teams vs. sport, min + max weight. 

# index = Sport
# columns = Team
# values = Weight
# aggfunc = ['min', 'max']

df.pivot_table(index='Sport',
              columns='Team',
              values='Weight',
              aggfunc=['min', 'max'])

min                                      max  \
Team                      China France India Israel United States  China   
Sport                                                                      
Alpine Skiing              60.0   47.0  54.0   74.0          53.0   75.0   
Athletics                  42.0   43.0  43.0   44.0          41.0  120.0   
Badminton                  52.0   55.0  59.0   61.0          52.0   81.0   
Biathlon                   52.0   52.0   NaN    NaN          48.0   71.0   
Bobsleigh                   NaN   70.0   NaN    NaN           NaN    NaN   
Boxing                     48.0   48.0  48.0   48.0          48.0  111.0   
Canoeing                   55.0   53.0   NaN   64.0          55.0   92.0   
Cross Country Skiing       49.0   50.0  62.0    NaN          48.0   83.0   
Curling                    60.0   62.0   NaN    NaN          52.0   80.0   
Cycling                    51.0   47.0   NaN   58.0          48.0   90.0   
Diving                     28.0   48.0   NaN    NaN          40.0   69.0   
Equestrianism              58.0   48.0  60.0    NaN          48.0   73.0   
Fencing                    55.0   50.0   NaN   49.0          48.0  108.0   
Figure Skating             41.0   35.0   NaN   45.0          36.0   76.0   
Freestyle Skiing           46.0   48.0   NaN    NaN          48.0   75.0   
Gymnastics                 30.0   35.0  47.0   45.0          34.0   72.0   
Nordic Combined             NaN   59.0   NaN    NaN          59.0    NaN   
Rhythmic Gymnastics        40.0   30.0   NaN   44.0          40.0   57.0   
Rowing                     42.0   45.0  67.0    NaN          42.0   98.0   
Sailing                    55.0   49.0  60.0   50.0          52.0  100.0   
Shooting                   40.0   42.0  46.0   60.0          51.0  110.0   
Short Track Speed Skating  53.0   46.0   NaN   84.0          50.0   78.0   
Ski Jumping                56.0   51.0   NaN    NaN          44.0   66.0   
Snowboarding               48.0   50.0   NaN    NaN          48.0   78.0   
Speed Skating              48.0   57.0   NaN    NaN          48.0   93.0   
Swimming                   50.0   52.0  58.0   52.0          48.0   98.0   
Synchronized Swimming      48.0   48.0   NaN   49.0          47.0  104.0   
Table Tennis               51.0   50.0  47.0   57.0          42.0   85.0   
Tennis                     55.0   54.0  57.0   52.0          55.0   85.0   
Trampolining               47.0   53.0   NaN    NaN          52.0   65.0   
Triathlon                  53.0   45.0   NaN   65.0          52.0   68.0   
Weightlifting              48.0   48.0  48.0   52.0          48.0  133.0   
Wrestling                  48.0   48.0  48.0   52.0          48.0  130.0   

                                                              
Team                      France  India Israel United States  
Sport                                                         
Alpine Skiing               98.0   71.0   74.0         102.0  
Athletics                  116.0  125.0   96.0         158.0  
Badminton                   80.0   85.0   61.0          91.0  
Biathlon                    80.0    NaN    NaN          84.0  
Bobsleigh                  123.0    NaN    NaN           NaN  
Boxing                     105.0   91.0   75.0         116.0  
Canoeing                    95.0    NaN   89.0          98.0  
Cross Country Skiing        81.0   74.0    NaN          86.0  
Curling                     80.0    NaN    NaN         102.0  
Cycling                    100.0    NaN   59.0         104.0  
Diving                      75.0    NaN    NaN          80.0  
Equestrianism              100.0   70.0    NaN          88.0  
Fencing                    100.0    NaN   80.0          98.0  
Figure Skating              82.0    NaN   84.0          86.0  
Freestyle Skiing            92.0    NaN    NaN          98.0  
Gymnastics                  74.0   47.0   77.0          75.0  
Nordic Combined             80.0    NaN    NaN          81.0  
Rhythmic Gymnastics         55.

In [53]:
# 4. Show a pivot table of teams vs. sport, min and max weight + height.

# index = 'Sport'
# columns = Team
# values = ['Weight', 'Height']
# aggfunc = ['min', 'max']

df.pivot_table(index='Sport',
              columns='Team',
              values=['Weight', 'Height'],
              aggfunc=['min', 'max'])

min                                            \
                          Height                                    Weight   
Team                       China France  India Israel United States  China   
Sport                                                                        
Alpine Skiing              160.0  154.0  162.0  181.0         160.0   60.0   
Athletics                  155.0  155.0  152.0  148.0         150.0   42.0   
Badminton                  161.0  164.0  160.0  171.0         157.0   52.0   
Biathlon                   160.0  160.0    NaN    NaN         152.0   52.0   
Bobsleigh                    NaN  167.0    NaN    NaN           NaN    NaN   
Boxing                     165.0  158.0  158.0  165.0         155.0   48.0   
Canoeing                   160.0  160.0    NaN  165.0         157.0   55.0   
Cross Country Skiing       159.0  155.0  155.0    NaN         157.0   49.0   
Curling                    162.0  174.0    NaN    NaN         155.0   60.0   
Cycling                    160.0  155.0    NaN  162.0         155.0   51.0   
Diving                     137.0  159.0    NaN    NaN         150.0   28.0   
Equestrianism              168.0  156.0  158.0    NaN         152.0   58.0   
Fencing                    164.0  162.0    NaN  161.0         157.0   55.0   
Figure Skating             153.0  148.0    NaN  152.0         147.0   41.0   
Freestyle Skiing           157.0  152.0    NaN    NaN         152.0   46.0   
Gymnastics                 136.0  140.0  151.0  155.0         139.0   30.0   
Nordic Combined              NaN  171.0    NaN    NaN         167.0    NaN   
Rhythmic Gymnastics        160.0  150.0    NaN  157.0         155.0   40.0   
Rowing                     150.0  150.0  180.0    NaN         142.0   42.0   
Sailing                    162.0  158.0  167.0  160.0         157.0   55.0   
Shooting                   152.0  146.0  157.0  167.0         155.0   40.0   
Short Track Speed Skating  160.0  156.0    NaN  186.0         152.0   53.0   
Ski Jumping                166.0  163.0    NaN    NaN         160.0   56.0   
Snowboarding               158.0  160.0    NaN    NaN         157.0   48.0   
Speed Skating              158.0  160.0    NaN    NaN         157.0   48.0   
Swimming                   162.0  163.0  163.0  159.0         160.0   50.0   
Synchronized Swimming      162.0  158.0    NaN  158.0         155.0   48.0   
Table Tennis               150.0  157.0  149.0  167.0         157.0   51.0   
Tennis                     164.0  165.0  165.0  159.0         160.0   55.0   
Trampolining               155.0  164.0    NaN    NaN         154.0   47.0   
Triathlon                  165.0  153.0    NaN  175.0         157.0   53.0   
Weightlifting              148.0  150.0  145.0  156.0         150.0   48.0   
Wrestling                  154.0  157.0  152.0  151.0         150.0   48.0   

                                                               max         \
                                                            Height          
Team                      France India Israel United States  China France   
Sport                                                                       
Alpine Skiing               47.0  54.0   74.0          53.0  180.0  191.0   
Athletics                   43.0  43.0   44.0          41.0  202.0  198.0   
Badminton                   55.0  59.0   61.0          52.0  190.0  186.0   
Biathlon                    52.0   NaN    NaN          48.0  175.0  188.0   
Bobsleigh                   70.0   NaN    NaN           NaN    NaN  198.0   
Boxing                      48.0  48.0   48.0          48.0  200.0  200.0   
Canoeing                    53.0   NaN   64.0          55.0  194.0  203.0   
Cross Country Skiing        50.0  62.0    NaN          48.0  188.0  186.0   
Curling                     62.0   NaN    NaN          52.0  185.0  183.0   
Cycling                     47.0   NaN   58.0          48.0  189.0  192.0   
Diving                      48.0   NaN    NaN          40.0  172

In [57]:
# 5. Show a pivot table of years vs. teams, number of competitors.
# Retrieve the number for Israel in each year.

# index = Year
# column = Team
# values = Name
# aggfunc = count

df.pivot_table(index='Year',
              columns='Team',
              values='Name',
              aggfunc='count').fillna(0).astype(int)

Team,China,France,India,Israel,United States
Year,,,,,
1980,38,230,36,0,106
1984,312,367,37,63,642
1988,348,452,36,29,712
1992,394,542,39,40,766
1994,43,131,0,1,187
1996,320,385,28,36,611
1998,68,123,0,3,212
2000,296,381,54,55,572
2002,83,138,0,3,231


In [60]:
# 5. Show a pivot table of years vs. teams, number of competitors.
# Retrieve the number for Israel in each year.

# index = Year
# column = Team
# values = Name
# aggfunc = count

df.pivot_table(index='Year',
              columns='Team',
              values='Name',
              aggfunc=['min', 'max'])

min                              \
Team           China                      France   
Year                                               
1980    Bao Zhenghua                Alain Bondue   
1984    Bao Zhenghua  Agns Gosselin (-Tavernier)   
1988    Cai Jianming  Agns Gosselin (-Tavernier)   
1992  Bai Chongguang   Abdelghani "Ghani" Yalouz   
1994         Chen Lu                Alexis Blanc   
1996     Ba Yanchuan        Abdel Kader Chkhmani   
1998       An Yulong            Adrien Duvillard   
2000   Abudoureheman                Abdel Jebahi   
2002       An Yulong         Alexandre Rousselet   
2004       Ai Linuer        Abderrahim El Haouzy   
2006     An Weijiang         Alexandre Rousselet   
2008         Bai Xue                Adrien Hardy   
2010     Cai Xuetong                Adrien Thaux   
2012        Bai Anqi           Abdellatif Meftah   
2014        Ba Dexin     Adeline Baud (-Mugnier)   
2016       Ai Yanhan                 Adrien Bart   

                                                                     \
Team                               India                     Israel   
Year                                                                  
1980                  Adille Sumariwalla                        NaN   
1984                 Baljit Singh Kharab               Arie Gamliel   
1988  Alapurackal "Mercy" Kuttan-Mathews        Aharon Jacobashvili   
1992                        Abha Dhillan            Aleksey Bazarov   
1994                                 NaN           Michael Shmerkin   
1996                      Ambika Radhika             Alex Tripolski   
1998                                 NaN                Galit Chait   
2000                      Abhinav Bindra           Adi Maia Bichman   
2002                                 NaN               Olga Danilov   
2004                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2006                Bahadur Gurung Gupta             Mikail Renzhin   
2008                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2010                     Jamyang Namgial         Alexandra Zaretski   
2012                      Abhinav Bindra         Aleksandr Shatilov   
2014                     Himanshu Thakur  Alexei Yuryevich Bychenko   
2016                      Abhinav Bindra               Ageze Guadie   

                                                   max                        \
Team                      United States          China                France   
Year                                                                           
1980  Abigail E. "Abbi" Fisher (-Gould)  Zhao Weichang           Yvon Mougel   
1984                    Abdurrahim Kuzu   Zou Zhenxian         velyne Imbert   
1988  Abigail Knickerbocker "Abby" Peck    Zhuang Yong           velyne lien   
1992                   Aaron E. Pollock      Zou Sixin  ric Paul Alain Navet   
1994           Alva Ross "A J" Kitt, IV     Zhao Guona           tienne Gouy   
1996           Adam Christophe Saathoff      Zou Sixin           velyne lien   
1998                     Adam Hostetter    Zhao Hongbo        ric Le Chanony   
2000                Aaron Wells Peirsol         Zhu Yi          ric le Leuch   
2002                     Aelin Peterson  Zhang Xiaolei        Vincent Vittoz   
2004                Aaron Wells Peirsol    Zou Shiming  ric Paul Alain Navet   
2006         Abigail Jean "Abby" Larson  Zhang Zhongqi            milie Vina   
2008                       Aarik Wilson    Zou Shiming           lodie Gugan   
2010                     Alice McKennis      Zhou Yang            milie Vina   
2012        Abdihakim "Abdi" Abdirahman    Zou Shiming         tienne Daille   
2014                       Aaron Blunck      Zhou Yang        Xavier Bertoni   
2016                     Abbey Weitzeil       Zu Lijun         tienne Hubert   

                                                             \
Team                              India              Israel   
Year                           

In [63]:
# get data for Israel

pt = df.pivot_table(index='Year',
              columns='Team',
              values='Name',
              aggfunc=['min', 'max'])

pt.xs('Israel', level='Team', axis='columns')

,min,max
Year,,
1980,NaN,NaN
1984,Arie Gamliel,Zehava Shmueli
1988,Aharon Jacobashvili,Yoel Sela
1992,Aleksey Bazarov,Yoel Sela
1994,Michael Shmerkin,Michael Shmerkin
1996,Alex Tripolski,Yoav Bruck
1998,Galit Chait,Sergey Sakhnovsky
2000,Adi Maia Bichman,Yuriy Yevseychyk
2002,Olga Danilov,Olga Danilov
